In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np


In [ ]:
output_folder_path=r"E:\VSC_SSD\prova_out" # path to save images and final results


root_folder = r"E:\VSC_SSD\MicroNeuSeg\results" # Define the folder containing all data files (savepath in the 4_extract_data.py)

# Create an empty list to store DataFrames
data_frames = []

# Walk through all subdirectories and files
for dirpath, dirnames, filenames in os.walk(root_folder):
    for file in filenames:
        if file.endswith(".xlsx"):  # Check if the file is an Excel file
            # Extract the part of the filename before "_PARVA"
            identifier = file.split(".")[0]
            
            # Read the Excel file
            file_path = os.path.join(dirpath, file)
            df = pd.read_excel(file_path)
            
            # Add the identifier as a new column
            df["Identifier"] = identifier
            df["Sex"] = df["Identifier"].str.extract(r"_(FEM|M)_")
            df["Genotype"] = df["Identifier"].str.extract(r"_(WT|TS)_")
            df["Animal"] = df["Identifier"].str.replace(r"_[0-9]+$", "", regex=True)
            
            # Append the DataFrame to the list
            data_frames.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)
combined_df["Genotype"] = combined_df['Genotype'].replace({'WT': 'Eu', 'TS': 'Ts'})
combined_df["Batch"] = combined_df["Animal"].apply(lambda x: 1 if "PG" in x else 2)
combined_df["E_I_Ratio"]=(combined_df["Percent_Microglia_Associated_with_NeuN"]-combined_df["Percent_Microglia_Associated_with_PV"])/(combined_df["Percent_Microglia_Associated_with_NeuN"]+combined_df["Percent_Microglia_Associated_with_PV"])
combined_df.drop(columns=["Area", "NeuN+PV-_Density"], axis=1,inplace=True)
combined_df["E_I_Ratio"].replace([np.inf, -np.inf], np.nan, inplace=True)
combined_df.dropna(subset=["E_I_Ratio"], inplace=True)
# Save the combined DataFrame to a new file (optional)


combined_df.head()


In [ ]:
combined_df.columns

In [ ]:
#combined_df.sort_values(by=["Genotype", "Sex"]).to_excel(os.path.join(output_folder_path, "Figure3_all_data_proximity_1.xlsx"), index=False)

# excluding outliers - optional

In [ ]:
combined_df['Animal'].unique()

# EXPLORATORY DATA ANALYSIS USING LLM AND ANCOVA on SINGLE SLICES

In [ ]:
# categorical_cols = ['Identifier', 'Sex', 'Genotype', 'Animal']
# combined_df.loc[:, categorical_cols] = combined_df.loc[:, categorical_cols].astype('category')
# combined_df['Genotype'].cat.categories # the first cat will be used as reference in the LMM
# combined_df['Sex'].cat.categories # the first cat will be used as reference in the LMM


# ts = combined_df[combined_df["Genotype"]=="Ts"]
# wt = combined_df[combined_df["Genotype"]=="Eu"]
# print(wt["Percent_Associated_PV"].mean() - ts["Percent_Associated_PV"].mean())
# print(wt["Percent_Associated_NeuN"].mean() - ts["Percent_Associated_NeuN"].mean())

# let's make a linear mixed model effect that truly rules out effect of slice

In [ ]:
# Placeholder for results
results = []


metrics = ['Microglia_Density', 'PV_Density', 'NeuN_Density',
       'Percent_Associated_PV', 'Proximity_MEAN_PV_BOOTSTRAP',
       'Average_Nearest_Distance_PV', 'Average_Nearest_Distance_PV_BOOT',
       'Percent_Associated_NeuN', 'Proximity_MEAN_NEUN_BOOTSTRAP',
       'Average_Nearest_Distance_NeuN', 'Average_Nearest_Distance_NeuN_BOOT',
       'Percent_Microglia_Associated_with_PV',
       'Proximity_MEAN_PVMICRO_BOOTSTRAP',
       'Average_Nearest_Distance_PV-ass MICRO',
       'Average_Nearest_Distance_PVMICRO_BOOT',
       'Percent_Microglia_Associated_with_NeuN',
       'Proximity_MEAN_NEUNMICRO_BOOTSTRAP',
       'Average_Nearest_Distance_NeuN-ass MICRO',
       'Average_Nearest_Distance_NeuNMICRO_BOOT', "E_I_Ratio"]



# Perform LMM for each metric
for metric in metrics:
    # Define the fixed effects formula (random effects are defined separately)
    formula = f"{metric} ~ C(Genotype) * C(Sex)"
    
    try:
        # Drop missing values for the current metric
        df_filtered = combined_df.dropna(subset=[metric, "Genotype", "Sex", "Animal"])

        # Fit the linear mixed model with a random intercept for "Animal" and use an alternative optimizer
        model = smf.mixedlm(formula, data=df_filtered, groups=df_filtered["Animal"]).fit(method="powell")

        # Extract fixed effect p-values
        p_values = model.pvalues
        available_terms = list(p_values.index)  # Log available terms
        # print(f"Available terms for {metric}: {available_terms}") # optional: printing available terms

        # Extract p-values for fixed effects dynamically
        p_value_genotype = p_values.get("C(Genotype)[T.Ts]", "N/A")
        p_value_sex = p_values.get("C(Sex)[T.M]", "N/A")
        p_value_interaction = p_values.get("C(Genotype)[T.Ts]:C(Sex)[T.M]", "N/A")

        # Check if the model converged
        converged = model.converged

        # Append results
        results.append({
            "Metric": metric,
            "P-Value (Genotype)": p_value_genotype,
            "P-Value (Sex)": p_value_sex,
            "P-Value (Interaction)": p_value_interaction,
            "Significance (Genotype)": p_value_genotype < 0.05 if isinstance(p_value_genotype, (float, int)) else "N/A",
            "Significance (Sex)": p_value_sex < 0.05 if isinstance(p_value_sex, (float, int)) else "N/A",
            "Significance (Interaction)": p_value_interaction < 0.05 if isinstance(p_value_interaction, (float, int)) else "N/A",
            #"AIC": model.aic,
            "Converged": converged
        })

    except Exception as e:
        print(f"Error fitting LMM for {metric}: {e.__class__.__name__} - {e}")

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to Excel
output_file = "Table S3.xlsx"
#results_df.to_excel(os.path.join(output_folder_path, output_file), index=False)

results_df

In [ ]:
palette_true= ["darkcyan", "lightpink"]
condition_order = ["Eu", "Ts"]

# Assuming df is your DataFrame
plt.figure(figsize=(2, 3))
sns.boxplot(x='Genotype', y='Percent_Associated_PV', data=combined_df, palette=palette_true)
sns.stripplot(x='Genotype', y='Percent_Associated_PV', data=combined_df, color='black', alpha=0.5)
#plt.title('Average Nearest Distance PV(um) per Genotype')
plt.ylabel('Microglia Percent_Associated_PV (n/mm2)')
plt.xlabel('')
sns.despine()

#plt.savefig(os.path.join(output_folder_path, "Microglia_Density.svg"), bbox_inches="tight")

In [ ]:
# Assuming df is your DataFrame
plt.figure(figsize=(2, 3))
sns.boxplot(x='Genotype', y='PV_Density', data=combined_df, palette=palette_true)
sns.stripplot(x='Genotype', y='PV_Density', data=combined_df, color='black', alpha=0.5)
#plt.title('Average Nearest Distance PV(um) per Genotype')
plt.ylabel('PV Density (n/mm2)')
plt.xlabel('')
sns.despine()
#plt.savefig(os.path.join(output_folder_path, "PV_Density.svg"), bbox_inches="tight")

In [ ]:
# Assuming df is your DataFrame
plt.figure(figsize=(2, 3))
sns.boxplot(x='Genotype', y='Percent_Microglia_Associated_with_PV', data=combined_df, palette=palette_true)
sns.stripplot(x='Genotype', y='Percent_Microglia_Associated_with_PV', data=combined_df, color='black', alpha=0.5)
#plt.title('Average Nearest Distance PV(um) per Genotype')
plt.ylabel('Percent_Microglia_Associated_with_PV')
plt.xlabel('')
sns.despine()
#plt.savefig(os.path.join(output_folder_path, "Percent_Microglia_Associated_with_PV.svg"), bbox_inches="tight")

In [ ]:
# Assuming df is your DataFrame
plt.figure(figsize=(2, 3))
sns.boxplot(x='Genotype', y='Average_Nearest_Distance_PV', data=combined_df, palette=palette_true)
sns.stripplot(x='Genotype', y='Average_Nearest_Distance_PV', data=combined_df, color='black', alpha=0.5)
#plt.title('Average Nearest Distance PV(um) per Genotype')
plt.ylabel('Average Nearest Distance PV (um)')
plt.xlabel('')
sns.despine()
#plt.savefig(os.path.join(output_folder_path, "Average_Nearest_Distance_PV.svg"), bbox_inches="tight")

In [ ]:
# Assuming df is your DataFrame
plt.figure(figsize=(2, 3))
sns.boxplot(x='Genotype', y='Average_Nearest_Distance_NeuN', data=combined_df, palette=palette_true)
sns.stripplot(x='Genotype', y='Average_Nearest_Distance_NeuN', data=combined_df, color='black', alpha=0.5)
#plt.title('Average Nearest Distance PV(um) per Genotype')
plt.ylabel('Average Nearest Distance NeuN (um)')
plt.xlabel('')
sns.despine()
#plt.savefig(os.path.join(output_folder_path, "Average_Nearest_Distance_NeuN.svg"), bbox_inches="tight")

In [ ]:
# Assuming df is your DataFrame
plt.figure(figsize=(2, 3))
sns.boxplot(x='Genotype', y='Average_Nearest_Distance_PV_BOOT', data=combined_df, palette=palette_true)
sns.stripplot(x='Genotype', y='Average_Nearest_Distance_PV_BOOT', data=combined_df, color='black', alpha=0.5)
#plt.title('Average Nearest Distance PV(um) per Genotype')
plt.ylabel('Average Nearest Distance PV (bootstrap)')
plt.xlabel('')
sns.despine()
#plt.savefig(os.path.join(output_folder_path, "Average_Nearest_Distance_PV_BOOT.svg"), bbox_inches="tight")

In [ ]:
# Assuming df is your DataFrame
plt.figure(figsize=(2, 3))
sns.boxplot(x='Genotype', y='Average_Nearest_Distance_NeuN_BOOT', data=combined_df, palette=palette_true)
sns.stripplot(x='Genotype', y='Average_Nearest_Distance_NeuN_BOOT', data=combined_df, color='black', alpha=0.5)
#plt.title('Average Nearest Distance PV(um) per Genotype')
plt.ylabel('Average Nearest Distance NeuN (bootstrap)')
plt.xlabel('')
sns.despine()
plt.savefig(os.path.join(output_folder_path, "Average_Nearest_Distance_NeuN_BOOT.svg"), bbox_inches="tight")

In [ ]:
palette_true= ["darkcyan", "lightpink"]
condition_order = ["WT", "TS"]

# Assuming df is your DataFrame
plt.figure(figsize=(4, 4))
sns.boxplot(x='Genotype', y='Average_Nearest_Distance_PV', data=combined_df, palette=palette_true)
sns.stripplot(x='Genotype', y='Average_Nearest_Distance_PV', data=combined_df, color='black', alpha=0.5)

#plt.title('Average Nearest Distance PV(um) per Genotype')
plt.ylabel('Average Nearest Distance PV (μm)')

sns.despine()

In [ ]:
# Select numeric columns, include 'Animal', and compute grouped means
grouped_means = (
    combined_df.select_dtypes(include=['number'])
    .assign(Animal=combined_df['Animal'])
    .groupby('Animal')
    .mean()
)
grouped_means = grouped_means.reset_index()

# Step 1: Extract genotype from the 'Animal' column (last part of the name)
grouped_means['Genotype'] = grouped_means['Animal'].str.extract(r"_(WT|TS)")

# Step 2: Separate data by genotype
genotype_groups = grouped_means.groupby('Genotype')

# Step 3: Perform t-tests for all numeric columns
# Exclude non-numeric columns
numeric_cols = grouped_means.select_dtypes(include=['number']).columns
results = []

for col in numeric_cols:
    if col != 'Animal':  # Skip 'Animal' column if it's numeric
        group1 = genotype_groups.get_group('WT')[col]  # Replace 'WT' with your genotype name
        group2 = genotype_groups.get_group('TS')[col]  # Replace 'Ts' with your genotype name
        t_stat, p_val = ttest_ind(group1, group2, equal_var=False)  # Welch's t-test
        results.append({
            'Metric': col,
            'T-Statistic': t_stat,
            'P-Value': p_val,
            'Significant (p<0.05)': p_val < 0.05
        })

# Step 4: Convert results to a DataFrame
t_test_results = pd.DataFrame(results)
t_test_results
# Step 5: Save results to an Excel file (optional)
#t_test_results.to_excel('t_test_results_by_genotype.xlsx', index=False)

In [ ]:
grouped_means['Genotype']=grouped_means['Animal'].str.extract(r"_(WT|TS)")

In [ ]:
grouped_means.sort_values(by="Genotype").to_excel(os.path.join(output_folder_path, "Grouped_data_all.xlsx"), index=False) # saving the entire DF